In [85]:
# !pip install pyproj

In [86]:
import requests as rq
import urllib.parse as parse
import urllib.request as request
import pandas as pd
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np
import pyproj
# import folium

In [87]:
urls = pd.read_csv("./data/에코뱅크_url모음_수정.csv")
urls.columns = ['service', 'url']
urls.head()

,service,url
0,백두대간_곤충_점,/ecoapi/BgtsInfoService/wfs/getInsectPointWFS
1,백두대간_식물상_점,/ecoapi/BgtsInfoService/wfs/getFlrPointWFS
2,백두대간_식생_면,/ecoapi/BgtsInfoService/wfs/getVtnPynWFS
3,백두대간_양서파충류_점,/ecoapi/BgtsInfoService/wfs/getAmnrpPotinWFS
4,백두대간_어류_점,/ecoapi/BgtsInfoService/wfs/getFishesPointWFS


In [88]:
# service_dict = {}
# col_temp = []
# val_temp = []
# for col in df["service"]:
#     col_temp.append(col)
# for val in df["url"]:
#     val_temp.append(val)
# for idx in range(len(col_temp)):
#     name = col_temp[idx]
#     service_dict[name] = val_temp[idx]

# print(service_dict)

In [89]:
layers = pd.DataFrame()

layers['service'] = [
    '백두대간_곤충_점',
    '백두대간_식물상_점',
    '백두대간_양서파충류_점',
    '백두대간_어류_점',
    '백두대간_저서무척추동물_점',
    '백두대간_조류_점',
    '백두대간_지의류_점',
    '백두대간_지형경관_점',
    '백두대간_포유류_점'
]

layers['layer'] = [
    'mv_map_bgts_insect_point',
    'mv_map_bgts_flr_point',
    'mv_map_bgts_amnrp_point',
    'mv_map_bgts_fishes_point',
    'mv_map_bgts_bnin_point',
    'mv_map_bgts_birds_point',
    'mv_map_bgts_lchn_point	',
    'mv_map_bgts_tpsc_point',
    'mv_map_bgts_mml_point'

]

layers

,service,layer
0,백두대간_곤충_점,mv_map_bgts_insect_point
1,백두대간_식물상_점,mv_map_bgts_flr_point
2,백두대간_양서파충류_점,mv_map_bgts_amnrp_point
3,백두대간_어류_점,mv_map_bgts_fishes_point
4,백두대간_저서무척추동물_점,mv_map_bgts_bnin_point
5,백두대간_조류_점,mv_map_bgts_birds_point
6,백두대간_지의류_점,mv_map_bgts_lchn_point\t
7,백두대간_지형경관_점,mv_map_bgts_tpsc_point
8,백두대간_포유류_점,mv_map_bgts_mml_point


In [90]:
layer_list = pd.merge(layers, urls)

In [91]:
layer_list

,service,layer,url
0,백두대간_곤충_점,mv_map_bgts_insect_point,/ecoapi/BgtsInfoService/wfs/getInsectPointWFS
1,백두대간_식물상_점,mv_map_bgts_flr_point,/ecoapi/BgtsInfoService/wfs/getFlrPointWFS
2,백두대간_양서파충류_점,mv_map_bgts_amnrp_point,/ecoapi/BgtsInfoService/wfs/getAmnrpPotinWFS
3,백두대간_어류_점,mv_map_bgts_fishes_point,/ecoapi/BgtsInfoService/wfs/getFishesPointWFS
4,백두대간_저서무척추동물_점,mv_map_bgts_bnin_point,/ecoapi/BgtsInfoService/wfs/getBninPointWFS
5,백두대간_조류_점,mv_map_bgts_birds_point,/ecoapi/BgtsInfoService/wfs/getBirdsPointWFS
6,백두대간_지의류_점,mv_map_bgts_lchn_point\t,/ecoapi/BgtsInfoService/wfs/getIchnPointWFS
7,백두대간_지형경관_점,mv_map_bgts_tpsc_point,/ecoapi/BgtsInfoService/wfs/getTpscPointWFS
8,백두대간_포유류_점,mv_map_bgts_mml_point,/ecoapi/BgtsInfoService/wfs/getMmlPointWFS


# Code

In [92]:
def api_parser(serviceUrl, typeName):
    url = "https://www.nie-ecobank.kr{}".format(serviceUrl)
    service_key = "72NKW00P8C39C6VG5I2D1RK62UX7627S2755T22N"
    queryParams = '?' + parse.urlencode({ parse.quote_plus('serviceKey', encoding='UTF-8') : service_key,
                                      parse.quote_plus('srs', encoding='UTF-8') : "EPSG:5186",
                                      parse.quote_plus('bbox', encoding='UTF-8') : '-198374.40925553057, 52241.4120596305, 731236.4637184031, 703384.3127161775',
                                      parse.quote_plus('typeName', encoding='UTF-8') : typeName,
                                      parse.quote_plus('maxFeatures', encoding='UTF-8') : "500"
                                      })
    queryParams = parse.unquote_plus(queryParams, encoding="UTF-8")
    full_url = url + queryParams
    res = rq.get(full_url)
#     print(res)
#     print(res.text)
    res.encoding = 'utf-8'
    return res

In [93]:
name = []
coordinates = []
latitude = []
longitude = []
examin_realm_se_code = []
examin_year = []
spcs_korean_nm = []
examin_area_nm = []
point_ln_pyn_se_code = []
examin_begin_de = []
examin_end_de = []

data = []

In [94]:
def convert_coord(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    
    return np.dstack([fx, fy])[0]

In [95]:
def collect_data(res_xml, name_param):

    # print(res_xml)
    xml_doc = ET.fromstring(res_xml)

    # name.append(name_param)

    for parent in xml_doc.iter("{http://www.opengis.net/gml}featureMember"):
        name.append(name_param)
        for child in parent:
            spcs_checked = False
            area_checked = False
            examin_begin_de_checked = False
            examin_end_de_checked = False
            for leaf in child:
                if "geom" in str(leaf):
                    for k in leaf:
                        for l in k:
                            if "coordinates" in str(l):
                                sub_l = (l.text).split(',')
                                coordinates.append(sub_l)
                                
                if "examin_realm_se_code" in str(leaf):
                    examin_realm_se_code.append(leaf.text)
                if "examin_year" in str(leaf):
                    examin_year.append(leaf.text)
                if "spcs_korean_nm" in str(leaf):
                    spcs_korean_nm.append(leaf.text)
                    spcs_checked = True
                if "examin_area_nm" in str(leaf):
                    examin_area_nm.append(leaf.text)
                    area_checked = True
                if "point_ln_pyn_se_code" in str(leaf):
                    point_ln_pyn_se_code.append(leaf.text)
                if "examin_begin_de" in str(leaf):
                    examin_begin_de.append(leaf.text)
                    examin_begin_de_checked = True
                if "examin_end_de" in str(leaf):
                    examin_end_de.append(leaf.text)
                    examin_end_de_checked = True
            if not spcs_checked:
                spcs_korean_nm.append(None)
            if not area_checked:
                examin_area_nm.append(None)
            if not examin_begin_de_checked:
                examin_begin_de.append(None)
            if not examin_end_de_checked:
                examin_end_de.append(None)

In [96]:
for i in range(len(layer_list)):
    globals()['res{}'.format(i)] = api_parser(layer_list.loc[i, 'url'], layer_list.loc[i, 'layer'])
    # print((eval('res%d'%i)).text) # 각 layer들에 대한 res.text 출력
    collect_data(eval('res%d'%i).text, layers.loc[i, 'service'])

In [97]:
for idx in range(len(coordinates)):
    coordinates[idx] = list(map(float, coordinates[idx]))

coordinates

[[335138.1472, 585846.4761],
 [333781.7576, 587933.5936],
 [330416.4047, 587007.213],
 [276906.8834, 363403.8866],
 [369622.247, 513510.9765],
 [312386.9763, 645183.947],
 [334370.3612, 585621.3734],
 [328969.3411, 586213.3081],
 [356081.6978, 493556.8913],
 [332775.3593, 600739.2103],
 [329240.628, 605401.4784],
 [372369.6019, 492370.4966],
 [361103.8066, 496452.4479],
 [357279.0546, 495072.4281],
 [330416.4047, 587007.213],
 [333358.268, 590570.5304],
 [329240.628, 605401.4784],
 [312386.9763, 645183.947],
 [333781.7576, 587933.5936],
 [356386.1514, 559672.7448],
 [369457.7322, 557097.1148],
 [376909.9598, 535467.5206],
 [376909.9598, 535467.5206],
 [320308.1333, 637309.9551],
 [372369.6019, 492370.4966],
 [362081.0868, 493738.7464],
 [367497.7133, 494067.8363],
 [357666.579, 494615.7763],
 [320308.1333, 637309.9551],
 [377283.1817, 519118.1471],
 [374199.9738, 545846.3226],
 [371492.4445, 551990.4444],
 [358050.0171, 561628.1609],
 [319405.5785, 626748.3922],
 [333781.7576, 587933.5

In [98]:
coordinates = convert_coord(np.array(coordinates), 'epsg:5186', 'epsg:4326')
coordinates

C:\Users\samsung\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\samsung\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\samsung\AppData\Local\Temp/ipykernel_16160/4037489056.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  f

array([[128.5356806 ,  37.86246944],
       [128.5206583 ,  37.88146667],
       [128.48225   ,  37.87361111],
       ...,
       [128.26775   ,  38.34761111],
       [128.26775   ,  38.34761111],
       [128.38902778,  38.26488889]])

In [99]:
longitude = list(coordinates[:, 0])
latitude = list(coordinates[:, 1])

In [100]:
def deploy_aws_rds():
    print(len(coordinates))
    print(len(examin_realm_se_code))
    print(len(examin_year))
    print(len(spcs_korean_nm))
    print(len(examin_area_nm))
    print(len(point_ln_pyn_se_code))
    print(len(examin_begin_de))
    print(len(examin_end_de))
    print(len(name))

    df = pd.DataFrame({
        "서비스명": name,
#         "공간좌표": list(coordinates),
        "위도": latitude,
        "경도": longitude,
        "조사분야구분코드": examin_realm_se_code,
        "조사년도": examin_year,
        "종국문명칭": spcs_korean_nm,
        "조사지역명칭": examin_area_nm,
        "점선면구분코드": point_ln_pyn_se_code,
        "조사시작일자":examin_begin_de,
        "조사종료일자": examin_end_de
    })

    df = df.astype("string")
    print(df.dtypes)

    endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
    db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/dao'.format(endpoint)
    try:
        db_connection = create_engine(db_connection_str)
        conn = db_connection.connect()
    except:
        print('fail to connect db')

    df.to_sql(name='생태축_백두대간', con=db_connection, if_exists='replace', index=False)
    print("성공쓰~")

In [101]:
deploy_aws_rds()

4253
4253
4253
4253
4253
4253
4253
4253
4253
서비스명        string
위도          string
경도          string
조사분야구분코드    string
조사년도        string
종국문명칭       string
조사지역명칭      string
점선면구분코드     string
조사시작일자      string
조사종료일자      string
dtype: object
성공쓰~
